In [30]:
import pandas as pd
import numpy as np
from eep153_tools.sheets import read_sheets
Ethiopia_Data = '1qOZi1FqAS89x461pCIgwRaAfpBw5aUp7kBbp0tQKI_E'
# Change 'Ethiopia_Data' to key of your own sheet in Sheets, above
x = read_sheets(Ethiopia_Data,sheet = 'Food Expenditures (2011-12)')
x = x.set_index(['i','t','m','j']).squeeze()


# Now prices
p = read_sheets(Ethiopia_Data,sheet='Food Prices (2011-12)').set_index(['t','m','j','u'])

# Compute medians of prices for particular time, place and unit
p = p.groupby(['t','m','j','u']).median()

# Just keep metric units
p = p.xs('Kg',level="u").squeeze().unstack('j')

# Drop prices for goods we don't have expenditures for
# p = p[x.index.levels[-1]].T
p =  p.dropna()

# Household characteristics
d = read_sheets(Ethiopia_Data,sheet="Household Characteristics")
d.columns.name = 'k'

# Fill blanks with zeros
d = d.replace(np.nan,0)

# Expenditures x may have duplicate columns
x = x.T.groupby(['i','t','m','j']).sum()
x = x.replace(0,np.nan) # Replace zeros with missing

# Take logs of expenditures; call this y
y = np.log(x)

d.set_index(['i','t','m'],inplace=True)

In [32]:
d.groupby('t').mean()

k,Females 00-03,Females 04-08,Females 09-13,Females 14-18,Females 19-30,Females 31-50,Females 51-99,Males 00-03,Males 04-08,Males 09-13,Males 14-18,Males 19-30,Males 31-50,Males 51-99,log HSize
t,,,,,,,,,,,,,,,
2011-12,0.000000,0.332397,0.341843,0.272913,0.470258,0.424560,0.201174,0.000000,0.341843,0.368394,0.295124,0.406178,0.378095,0.246362,1.240192
2013-14,0.234645,0.345883,0.349306,0.319643,0.549724,0.436965,0.226849,0.249097,0.357102,0.354440,0.321354,0.467579,0.404640,0.229131,1.407929
2015-16,0.217172,0.340404,0.372727,0.289899,0.498788,0.487677,0.254747,0.240000,0.356364,0.375152,0.312929,0.420000,0.425051,0.207879,1.420437
2018-19,0.216248,0.320384,0.286263,0.280798,0.536780,0.428508,0.173855,0.224963,0.316987,0.293205,0.266470,0.424225,0.402659,0.186706,1.302566


In [34]:
from cfe import Regression

result = Regression(y=y, d=d)

AssertionError: Fewer rows than columns.  Consider passing the transpose.